In [4]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [5]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [6]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "actions" in dict.keys():
        matches = action_pattern.finditer(dict["actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_66_3.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

You arrive at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gather around the auctioneer, a worker carrying a bolt cutter snaps off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without giving any opportunity for the bidders to take a clear look, the auctioneer begins the bid at $50. You...

actions:
1. Bid $100
2. Bid $150
3. Bid $200


I select option 3. TURN: 1 


context: You confidently raise your hand, bidding $200. The room buzzes with whispers as some bidders shake their heads, stepping back, but a few others are still eyeing the lot, intrigued by your bold move. The auctioneer nods at you, then looks around for higher bids.

actions:
1. Bid $250 to outpace competition.
2. Hold your bid and watch the competition.
3. Ask questions about the storage's history.

I select option 2. TURN: 2 


context: Holding your bid at $200, you watch as two more bidders jump in, pushing the price up to $250. The auctioneer's eyes dart between the bidders and you, sensing the tension and the potential for a bidding war. The storage unit, with its mysterious contents, seems to beckon more interest with each passing second.

actions:
1. Bid $300 to take the lead.
2. Continue to hold and observe.
3. Drop out of the bidding.

I select option 1. TURN: 3 


context: With a determined nod, you bid $300, cutting through the growing murmur of the crowd. Your assertive move thins out the competition further, leaving only one other bidder who seems equally determined. The auctioneer's voice rises in excitement, announcing your bid and waiting for the next move.

actions:
1. Bid $350 if challenged.
2. Hold at $300 and observe the rival.
3. Engage the rival in a stare to intimidate.

I select option 1. TURN: 4 


context: As soon as your rival bids $325, without hesitation, you raise the stakes to $350. The crowd's chatter quiets down, all eyes on the two of you. The rival bidder hesitates, calculating, then finally, with a sigh, steps back. The auctioneer looks around for any more bids, the anticipation palpable in the air.

actions:
1. Prepare to celebrate your victory.
2. Offer a final look to encourage any last bids.
3. Ask the auctioneer to speed up the countdown.

I select option 2. TURN: 5 




RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}